In [6]:
import os
import pandas as pd
import sqlite3

In [7]:
data_directory = '/Users/kingsleyenweye/Desktop/INTELLIGENT_ENVIRONMENT_LAB/citylearn/data/nrel'
database_filepath = os.path.join(data_directory,'resstock.sql')

In [8]:
# data version we are interested in
dataset_type = 'resstock'
weather_data = 'tmy3'
year_of_publication = 2021
release_number = 1
root_url = f'https://oedi-data-lake.s3.amazonaws.com/nrel-pds-building-stock/end-use-load-profiles-for-us-building-stock/{year_of_publication}/{dataset_type}_{weather_data}_release_{release_number}/'

In [4]:
metadata = {
    'building_metadata':{
        'url':os.path.join(root_url,'metadata/metadata.parquet'),
        'reader':pd.read_parquet,
        'reader_kwargs':{}
    },
    'data_dictionary':{
        'url':os.path.join(root_url,'data_dictionary.tsv'),
        'reader':pd.read_csv,
        'reader_kwargs':{'sep':'\t'}
    },
    'enumeration_dictionary':{
        'url':os.path.join(root_url,'enumeration_dictionary.tsv'),
        'reader':pd.read_csv,
        'reader_kwargs':{'sep':'\t'}
    },
    'upgrade_dictionary':{
        'url':os.path.join(root_url,'upgrade_dictionary.tsv'),
        'reader':pd.read_csv,
        'reader_kwargs':{'sep':'\t'}
    },
    'geographic_metadata':{
        'url':os.path.join(root_url,'geographic_information/spatial_tract_lookup_table.csv'),
        'reader':pd.read_csv,
        'reader_kwargs':{'sep':','}
    },
}

for key, value in metadata.items():
    data = value['reader'](value['url'],**value['reader_kwargs'])
    metadata[key]['data'] = data.copy()

# Build SQLite DB
***

In [78]:
# building metadata
building_metadata = metadata['building_metadata']['data'].copy()
building_metadata = building_metadata.reset_index(drop=False)
building_metadata_datatypes = pd.DataFrame(building_metadata.dtypes,).reset_index()
building_metadata_datatypes.columns = ['field_name','data_type_pd']
building_metadata_datatypes['data_type_pd'] = building_metadata_datatypes['data_type_pd'].map(lambda x: x.name)
data_dictionary = metadata['data_dictionary']['data'].copy()
building_metadata_datatypes = pd.merge(building_metadata_datatypes,data_dictionary[['field_name','data_type']],on='field_name',how='left')
building_metadata_datatypes['data_type'] = building_metadata_datatypes['data_type'].combine_first(building_metadata_datatypes['data_type_pd'])
type_dict = {
    'REAL':['float64','float'],
    'INTEGER':['bool','integer','int64'],
    'TEXT':['string','object']
}

for key, value in type_dict.items():
    building_metadata_datatypes.loc[building_metadata_datatypes['data_type'].isin(value),'data_type_sqlite'] = key

building_metadata_columns = pd.merge(
    pd.DataFrame({'field_name':building_metadata.columns}),
    building_metadata_datatypes[['field_name','data_type','data_type_sqlite']],
    on='field_name',
    how='left'
)
building_metadata_columns['field_name_sqlite'] = building_metadata_columns['field_name'].map(lambda x: x.replace('.','_'))
queries = ['DROP TABLE IF EXISTS building_metadata;']
query = 'CREATE TABLE building_metadata ('

for (field_name_sqlite, data_type, data_type_sqlite) in building_metadata_columns[['field_name_sqlite','data_type','data_type_sqlite']].to_records(index=False):
    query += f'{field_name_sqlite} {data_type_sqlite},'

query += 'PRIMARY KEY (bldg_id));'
query = query.replace(',)',')')
queries.append(query)
con = sqlite3.connect(database_filepath)

for query in queries:
    con.execute(query)

building_metadata.columns = [c.replace('.','_') for c in building_metadata.columns]
building_metadata.to_sql(
    name='building_metadata',
    con=con,
    if_exists='append',
    index=False
)
con.commit()
display(pd.read_sql("SELECT * FROM building_metadata LIMIT 5",con))
con.close()

,bldg_id,in_county,in_puma,in_ashrae_iecc_climate_zone_2004,in_building_america_climate_zone,in_iso_rto_region,applicability,weight,in_sqft,in_ahs_region,...,out_propane_total_energy_consumption,out_propane_total_energy_consumption_intensity,out_wood_total_energy_consumption,out_wood_total_energy_consumption_intensity,out_wood_heating_energy_consumption,out_wood_heating_energy_consumption_intensity,out_site_energy_total_energy_consumption,out_site_energy_total_energy_consumption_intensity,upgrade,metadata_index
0,1,G5100230,G51051045,4A,Mixed-Humid,PJM,1,242.131013,1623.0,Non-CBSA South Atlantic,...,0.0,0.0,0.0,0.0,0.0,0.0,40030.480973,24.664498,0,0
1,2,G5600250,G56000400,6B,Cold,None,1,242.131013,617.0,Non-CBSA Mountain,...,0.0,0.0,0.0,0.0,0.0,0.0,26618.141083,43.141234,0,1
2,3,G4801130,G48002322,3A,Hot-Humid,ERCOT,1,242.131013,333.0,"CBSA Dallas-Fort Worth-Arlington, TX",...,0.0,0.0,0.0,0.0,0.0,0.0,9562.758683,28.716993,0,2
3,4,G2200190,G22000900,2A,Hot-Humid,MISO,1,242.131013,617.0,Non-CBSA West South Central,...,0.0,0.0,0.0,0.0,0.0,0.0,8744.273900,14.172243,0,3
4,5,G2901690,G29001400,4A,Mixed-Humid,None,1,242.131013,3241.0,Non-CBSA West North Central,...,0.0,0.0,0.0,0.0,0.0,0.0,18060.371136,5.572469,0,4


In [100]:
# data_dictionary
data_dictionary = metadata['data_dictionary']['data'].copy()
data_dictionary = pd.merge(
    data_dictionary,
    building_metadata_columns[['field_name_sqlite','field_name','data_type_sqlite']],
    how='left',
    on=['field_name']
)
data_dictionary_columns = pd.DataFrame(data_dictionary.dtypes,).reset_index()
data_dictionary_columns.columns = ['field_name','data_type_pd']
data_dictionary_columns['data_type_pd'] = data_dictionary_columns['data_type_pd'].map(lambda x: x.name)

for key, value in type_dict.items():
    data_dictionary_columns.loc[data_dictionary_columns['data_type_pd'].isin(value),'data_type_sqlite'] = key

data_dictionary_columns['field_name_sqlite'] = data_dictionary_columns['field_name'].map(lambda x: x.replace('.','_'))
queries = ['DROP TABLE IF EXISTS data_dictionary;']
query = 'CREATE TABLE data_dictionary (id INTEGER PRIMARY KEY NOT NULL,'

for (field_name_sqlite, data_type_sqlite) in data_dictionary_columns[['field_name_sqlite','data_type_sqlite']].to_records(index=False):
    query += f'{field_name_sqlite} {data_type_sqlite},'

query += ')'
query = query.replace(',)',')')
queries.append(query)
con = sqlite3.connect(database_filepath)

for query in queries:
    con.execute(query)

data_dictionary.columns = [c.replace('.','_') for c in data_dictionary.columns]
data_dictionary.to_sql(
    name='data_dictionary',
    con=con,
    if_exists='append',
    index=False
)
con.commit()
display(pd.read_sql("SELECT * FROM data_dictionary LIMIT 5",con))
con.close()

,id,field_location,field_name,data_type,units,field_description,allowable_enumerations,field_name_sqlite,data_type_sqlite
0,1,metadata,in.sqft,float,sqft,Finished floor area of the housing unit,317.0|328.0|333.0|617.0|633.0|853.0|866.0|885....,in_sqft,REAL
1,2,metadata,in.ahs_region,string,None,American Housing Survey region,"CBSA Atlanta-Sandy Springs-Roswell, GA|CBSA Bo...",in_ahs_region,TEXT
2,3,metadata,in.applicable,bool,None,Apply simulation output reporting measure,True|False,in_applicable,INTEGER
3,4,metadata,in.ashrae_iecc_climate_zone_2004,string,None,IECC climate zone,1A|2A|2B|3A|3B|3C|4A|4B|4C|5A|5B|6A|6B|7A|7B,in_ashrae_iecc_climate_zone_2004,TEXT
4,5,metadata,in.bathroom_spot_vent_hour,string,None,Bathroom spot ventilation daily start hour,Hour0|Hour1|Hour10|Hour11|Hour12|Hour13|Hour14...,in_bathroom_spot_vent_hour,TEXT


In [109]:
# building timeseries
query = """
SELECT
    bldg_id,
    in_county,
    upgrade
FROM building_metadata
WHERE
    in_geometry_building_type_recs IN ('Single-Family Detached')
    AND in_vacancy_status = 'Occupied'
    AND in_resstock_county_id IN ('TX, Travis County')
"""
con = sqlite3.connect(database_filepath)
buildings = pd.read_sql(query,con)

query = """
SELECT
    id,
    field_name_sqlite,
    data_type_sqlite
FROM data_dictionary
WHERE
    field_location = 'timeseries'
"""
building_timeseries_columns = pd.read_sql(query,con)
building_timeseries_columns = pd.concat([
        pd.DataFrame([
            {'field_name_sqlite':'timestamp','data_type_sqlite':'TEXT'},
            {'field_name_sqlite':'bldg_id','data_type_sqlite':'INTEGER'}
        ]),
        building_timeseries_columns,
    ],
    ignore_index=True,sort=False
)
queries = ['DROP TABLE IF EXISTS building_timeseries;']
query = 'CREATE TABLE building_timeseries ('

for (field_name_sqlite, data_type_sqlite) in building_timeseries_columns[['field_name_sqlite','data_type_sqlite']].to_records(index=False):
    query += f'{field_name_sqlite} {data_type_sqlite},'

query += 'PRIMARY KEY (timestamp, bldg_id), FOREIGN KEY (bldg_id) REFERENCES builging_metadata(bldg_id));'
query = query.replace(',)',')')
queries.append(query)
con = sqlite3.connect(database_filepath)

for query in queries:
    con.execute(query)

for i, ( bldg_id, county, upgrade) in enumerate(buildings.to_records(index=False)):
    url = os.path.join(root_url,f'timeseries_individual_buildings/by_county/upgrade={upgrade}/county={county}/{bldg_id}-{upgrade}.parquet')
    data = pd.read_parquet(url)
    data = data.reset_index(drop=False)
    data.columns = [c.replace('.','_') for c in data.columns]
    data.to_sql(
        name='building_timeseries',
        con=con,
        if_exists='append',
        index=False
    )
    con.commit()
    print(f'\rCompleted {i+1}/{len(buildings)}',end="")

print()
display(pd.read_sql("SELECT * FROM building_timeseries LIMIT 5",con))
con.close()

Completed 1020/1020


,timestamp,bldg_id,out_electricity_bath_fan_energy_consumption,out_electricity_bath_fan_energy_consumption_intensity,out_electricity_ceiling_fan_energy_consumption,out_electricity_ceiling_fan_energy_consumption_intensity,out_electricity_clothes_dryer_energy_consumption,out_electricity_clothes_dryer_energy_consumption_intensity,out_electricity_clothes_washer_energy_consumption,out_electricity_clothes_washer_energy_consumption_intensity,...,out_natural_gas_total_energy_consumption,out_natural_gas_total_energy_consumption_intensity,out_propane_total_energy_consumption,out_propane_total_energy_consumption_intensity,out_wood_total_energy_consumption,out_wood_total_energy_consumption_intensity,out_wood_heating_energy_consumption,out_wood_heating_energy_consumption_intensity,out_site_energy_total_energy_consumption,out_site_energy_total_energy_consumption_intensity
0,2018-01-01 00:15:00,40,0.000000,0.000000e+00,0.001892,2.309012e-07,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.531284,2.212372e-07
1,2018-01-01 00:30:00,40,0.007501,9.154065e-07,0.001892,2.309012e-07,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.718538,2.992133e-07
2,2018-01-01 00:45:00,40,0.007501,9.154065e-07,0.001892,2.309012e-07,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.865416,3.603759e-07
3,2018-01-01 01:00:00,40,0.007501,9.154065e-07,0.001892,2.309012e-07,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.964638,4.016942e-07
4,2018-01-01 01:15:00,40,0.007501,9.154065e-07,0.001892,2.309012e-07,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.110224,4.589946e-08


In [4]:
# add index on building id
query = "CREATE INDEX IF NOT EXISTS idx_bldg_id ON building_timeseries(bldg_id);"
con = sqlite3.caonnect(database_filepath)
con.execute(query)
con.commit()
con.close()